<a href="https://colab.research.google.com/github/9bishal/Learning_RAG/blob/main/RAG_Application_using_Langchain_Mistral_AI_and_Weviate_db_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install weaviate-client langchain tiktoken pypdf rapidocr-onnxruntime

In [2]:
WEAVIATE_CLUSTER="https://n2ok5qpfrqwecijxzx1lba.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY="M3NvM2hnaHZ3Zk9TR3lqNV9ldXk4RkZGVTFNaUxST2M5bG9qSzU1VWwzb25MR09MYStyUHBTbTBVUGRNPV92MjAw"

In [3]:
from langchain.vectorstores import Weaviate
import weaviate

In [ ]:
!pip install "weaviate-client>=3.26.7,<4.0.0"


In [5]:

WEAVIATE_URL=WEAVIATE_CLUSTER
WEAVIATE_API_KEY = WEAVIATE_API_KEY
client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret = weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

In [ ]:
!pip install -U langchain-community


In [7]:
import locale
locale.getpreferredencoding = lambda:"UTF-8"

In [ ]:
!pip install sentence-transformers

In [9]:
#specify embedding model
from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding_model_name="sentence-transformers/all-mpnet-base-v2"
# model_kwargs={"device":"cuda"}
embedding = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    # model_kwargs =model_kwargs
)

/tmp/ipython-input-893396662.py:5: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/document_for_RAG.pdf")
pages = loader.load()


In [11]:
# pages

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(pages)

In [13]:
# docs

In [14]:
# vector_db = Weaviate.from_documents(
#     docs, embedding, client=client, by_text=False

# )

In [15]:
def clean_property_names(doc):
    new_metadata = {}
    for key, value in doc.metadata.items():
        # Replace invalid characters (like dots) with underscores
        clean_key = key.replace(".", "_").replace("-", "_")
        new_metadata[clean_key] = value
    doc.metadata = new_metadata
    return doc

# Apply cleaning to all docs
docs = [clean_property_names(doc) for doc in docs]

# Now store them in Weaviate
vector_db = Weaviate.from_documents(
    docs, embedding, client=client, by_text=False
)


In [16]:
print(
    vector_db.similarity_search(
    "What is rag?", k=3
)[0].page_content
)

documents for questions that are less likely to beneﬁt from retrieval, suggesting that null document
mechanisms may not be necessary for RAG.
G Parameters
Our RAG models contain the trainable parameters for the BERT-base query and document encoder of
DPR, with 110M parameters each (although we do not train the document encoder ourselves) and
406M trainable parameters from BART-large, 406M parameters, making a total of 626M trainable
18


In [17]:
print(
    vector_db.similarity_search(
    "What is rag?", k=3
)[1].page_content
)

cases for NQ, where an extractive model would score 0%.
4.2 Abstractive Question Answering
As shown in Table 2, RAG-Sequence outperforms BART on Open MS-MARCO NLG by 2.6 Bleu
points and 2.6 Rouge-L points. RAG approaches state-of-the-art model performance, which is
impressive given that (i) those models access gold passages with speciﬁc information required to
generate the reference answer , (ii) many questions are unanswerable without the gold passages, and
(iii) not all questions are answerable from Wikipedia alone. Table 3 shows some generated answers
from our models. Qualitatively, we ﬁnd that RAG models hallucinate less and generate factually
correct text more often than BART. Later, we also show that RAG generations are more diverse than
BART generations (see §4.5).
4.3 Jeopardy Question Generation
Table 2 shows that RAG-Token performs better than RAG-Sequence on Jeopardy question generation,


In [18]:
print(
    vector_db.similarity_search(
    "What is rag?", k=3
)[2].page_content
)

and train RAG by directly minimizing the negative log-likelihood of answers. We compare RAG to
the popular extractive QA paradigm [5, 7, 31, 26], where answers are extracted spans from retrieved
documents, relying primarily on non-parametric knowledge. We also compare to “Closed-Book
QA” approaches [52], which, like RAG, generate answers, but which do not exploit retrieval, instead
relying purely on parametric knowledge. We consider four popular open-domain QA datasets: Natural
Questions (NQ) [29], TriviaQA (TQA) [24]. WebQuestions (WQ) [3] and CuratedTrec (CT) [2]. As
CT and WQ are small, we follow DPR [26] by initializing CT and WQ models with our NQ RAG
model. We use the same train/dev/test splits as prior work [ 31, 26] and report Exact Match (EM)
scores. For TQA, to compare with T5 [52], we also evaluate on the TQA Wiki test set.
3.2 Abstractive Question Answering
RAG models can go beyond simple extractive QA and answer questions with free-form, abstractive


In [19]:
from re import template
from langchain.prompts import ChatPromptTemplate
template="""You are an assisitant for question-answering task.
If you  don't know piece of retrieved context to answer the question.
Use ten sentencese maximum and keep the answer concise.
Question:{question}
Context :{context}
Answer:

"""

In [20]:
prompt =ChatPromptTemplate.from_template(template)

In [21]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assisitant for question-answering task.\nIf you  don't know piece of retrieved context to answer the question.\nUse ten sentencese maximum and keep the answer concise.\nQuestion:{question}\nContext :{context}\nAnswer:\n\n"), additional_kwargs={})])

In [22]:
from langchain_community.llms import HuggingFaceHub


In [30]:
from google.colab import userdata
huggingface_hub_api_token=userdata.get('HUGGINGFACE_TOKEN_FOR_RAG')

In [ ]:
model = HuggingFaceHub(
    huggingface_api_token = huggingface_hub_api_token,
    repo_id ="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs ={"temperature":1, "max_length":180}
)

In [33]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [34]:
output_parser=StrOutputParser()

In [35]:
retriever=vector_db.as_retriever()

In [38]:
rag_chain =(
    {"context":retriever, "question":RunnablePassthrough()}
    |prompt
    |model
    |output_parser
)

In [ ]:
rag_chain.invoke("what us RAG?")